# Copyright Netherlands eScience Center <br>
** Function     : Packing netCDF for land mask from JRA55** <br>
** Author       : Yang Liu ** <br>
** First Built  : 2019.10.10 ** <br>
** Last Update  : 2019.10.12 ** <br>
Description     : This notebook aims to pack the land mask from JRA55.<br>
Return Values   : netCDF4 <br>
Caveat          : 

In [11]:
import numpy as np
from netCDF4 import Dataset
import pygrib
import os
import matplotlib.pyplot as plt

Initialization - Start with location of input and extraction of variables
Time span of each product:
- **ERA-Interim** 1979 - 2016
- **MERRA2**      1980 - 2016
- **JRA55**       1979 - 2015
- **ORAS4**       1958 - 2017
- **GLORYS2V3**   1993 - 2014
- **SODA3**       1980 - 2015

In [12]:
################################   Input zone  ######################################
# specify data path
# JRA55 2D fields - radiations
datapath = '/home/ESLT0068/WorkFlow/Core_Database_AMET_OMET_reanalysis/JRA55/regression/mask'
# specify output path for figures
output_path = '/home/ESLT0068/WorkFlow/Core_Database_AMET_OMET_reanalysis/JRA55/regression/mask'
####################################################################################

In [13]:
# benmark data and basic dimensions extraction
benchmark_grbs = pygrib.open(os.path.join(datapath,'anl_land.085_soilt.reg_tl319.199101_199112.grib'))
print ('Number of messages',benchmark_grbs.messages)
for messenger in benchmark_grbs:
    print (messenger)
benchmark_key = benchmark_grbs.message(1)
lats, lons = benchmark_key.latlons()
latitude = lats[:,0]
longitude = lons[0,:]

var = benchmark_key.values
print(var.shape)
mask_var = np.ma.getmask(var)
mask = np.ones(var.shape,dtype=int)
mask[mask_var==True] = 0
mask[mask_var==False] = 1

benchmark_grbs.close()

Number of messages 12
1:Soil Temperature:K (avgua):regular_gg:deepSoil:level 0:fcst time 0 hrs (avgua):from 199101010000
2:Soil Temperature:K (avgua):regular_gg:deepSoil:level 0:fcst time 0 hrs (avgua):from 199102010000
3:Soil Temperature:K (avgua):regular_gg:deepSoil:level 0:fcst time 0 hrs (avgua):from 199103010000
4:Soil Temperature:K (avgua):regular_gg:deepSoil:level 0:fcst time 0 hrs (avgua):from 199104010000
5:Soil Temperature:K (avgua):regular_gg:deepSoil:level 0:fcst time 0 hrs (avgua):from 199105010000
6:Soil Temperature:K (avgua):regular_gg:deepSoil:level 0:fcst time 0 hrs (avgua):from 199106010000
7:Soil Temperature:K (avgua):regular_gg:deepSoil:level 0:fcst time 0 hrs (avgua):from 199107010000
8:Soil Temperature:K (avgua):regular_gg:deepSoil:level 0:fcst time 0 hrs (avgua):from 199108010000
9:Soil Temperature:K (avgua):regular_gg:deepSoil:level 0:fcst time 0 hrs (avgua):from 199109010000
10:Soil Temperature:K (avgua):regular_gg:deepSoil:level 0:fcst time 0 hrs (avgua):from 

In [14]:
print(mask)

[[0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 [0 0 0 ... 0 0 0]
 ...
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]
 [1 1 1 ... 1 1 1]]


In [15]:
def pack_netcdf_point (mask, output_path):
    print ('*******************************************************************')
    print ('*********************** create netcdf file*************************')
    print ('*******************************************************************')
    #logging.info("Start creating netcdf file for the 2D fields of ERAI at each grid point.")
    # wrap the datasets into netcdf file
    # 'NETCDF3_CLASSIC', 'NETCDF3_64BIT', 'NETCDF4_CLASSIC', and 'NETCDF4'
    data_wrap = Dataset(os.path.join(output_path, 'JRA55_model_landsea_mask.nc'),'w',format = 'NETCDF4')
    # create dimensions for netcdf data
    lat_wrap_dim = data_wrap.createDimension('latitude',len(latitude))
    lon_wrap_dim = data_wrap.createDimension('longitude',len(longitude))
    # create coordinate variables for 1-dimensions
    lat_wrap_var = data_wrap.createVariable('latitude',np.float32,('latitude',))
    lon_wrap_var = data_wrap.createVariable('longitude',np.float32,('longitude',))
    # create the actual 4d variable
    mask_wrap_var = data_wrap.createVariable('mask',np.int32,('latitude','longitude'))
    # global attributes
    data_wrap.description = 'Landsea mask JRA55 on model grid'
    # variable attributes
    lat_wrap_var.units = 'degree_north'
    lon_wrap_var.units = 'degree_east'

    mask_wrap_var.units = '1'

    mask_wrap_var.long_name = 'land sea mask'

    # writing data
    lat_wrap_var[:] = latitude
    lon_wrap_var[:] = longitude

    mask_wrap_var[:] = mask

    # close the file
    data_wrap.close()
    print ("The generation of netcdf files for fields on surface is complete!!")

In [16]:
if __name__=="__main__":
    pack_netcdf_point(mask, output_path)
    print ('Packing netcdf files complete!')

print ("Create netcdf file successfully")

*******************************************************************
*********************** create netcdf file*************************
*******************************************************************
The generation of netcdf files for fields on surface is complete!!
Packing netcdf files complete!
Create netcdf file successfully
